# Proyecto Vehiculos

Este proyecto de ciencia de datos está enfocado en el análisis de datos provenientes de una página ficticia de venta de carros, denominada SaCar.com.

El objetivo principal es explorar y analizar el comportamiento del mercado de vehículos en la plataforma, identificando patrones relevantes en las características de los autos ofertados, los factores que influyen en su precio y el tiempo que permanecen publicados antes de ser vendidos. Para ello, se realizará una limpieza y preparación de los datos, seguido de un análisis exploratorio que permita responder preguntas clave y validar hipótesis sobre el funcionamiento del mercado en SaCar.com.

Este análisis busca aportar información valiosa tanto para vendedores como para compradores, facilitando la toma de decisiones y mejorando la experiencia de los usuarios en la plataforma.

In [23]:
# Import necessary libraries
import pandas as pd
import numpy as np  
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
from scipy import stats as st
from datetime import datetime
import math
from scipy.stats import levene

In [24]:
# Upload the dataset
vehicle_df = pd.read_csv('../vehicles_us.csv')
vehicle_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [25]:
vehicle_df

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


## Preparación de datos

In [26]:
# Check for Na values in the 'model_year' column
vehicle_df.fillna({'model_year': 2025}, inplace=True)
vehicle_df['model_year'] = vehicle_df['model_year'].astype(int)
print(vehicle_df['model_year'].isna().sum())

0


In [27]:
# Check for Na values in the 'cylinders' column
vehicle_df.fillna({'cylinders': 0}, inplace=True)
vehicle_df['cylinders'] = vehicle_df['cylinders'].astype(int)
print(vehicle_df['cylinders'].isna().sum())

0


In [28]:
# Check for Na values in the 'odometer' column
vehicle_df.fillna({'odometer': -1}, inplace=True)
print(vehicle_df['odometer'].isna().sum())

0


In [29]:
# Check for Na values in the 'paint_color' column
vehicle_df.fillna({'paint_color': 'unknown'}, inplace=True)
print(vehicle_df['paint_color'].isna().sum())

0


In [30]:
# Check for Na values in the 'is_4wd' column
vehicle_df.fillna({'is_4wd': 0}, inplace=True)
print(vehicle_df['is_4wd'].isna().sum())
vehicle_df.loc[vehicle_df['is_4wd'] == 1, 'is_4wd'] = True
vehicle_df.loc[vehicle_df['is_4wd'] == 0, 'is_4wd'] = False
vehicle_df['is_4wd'] = vehicle_df['is_4wd'].astype(bool)


0


C:\Users\Flia\AppData\Local\Temp\ipykernel_1216\3828673243.py:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [31]:
# Reconize the duplicated values
duplicados = vehicle_df[vehicle_df.duplicated(
    subset=['price', 'model_year', 'model', 'condition', 'odometer', 'fuel', 'transmission', 'paint_color'],
    keep=False
)]
df_drop_dupli = vehicle_df.drop_duplicates(
    subset=['price', 'model_year', 'model', 'condition', 'odometer', 'fuel', 'transmission', 'paint_color'],
    keep='first'
)
duplicados = duplicados.sort_values(
    by=['model_year', 'model', 'condition', 'odometer', 'fuel', 'transmission', 'paint_color', 'price', 'date_posted']
)

duplicados.head(10)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
10472,15900,1965,ford mustang,good,0,gas,60000.0,manual,convertible,black,False,2018-07-25,33
34997,15900,1965,ford mustang,good,8,gas,60000.0,manual,convertible,black,False,2018-10-10,17
42834,21000,1967,chevrolet malibu,excellent,8,gas,94000.0,automatic,convertible,blue,False,2018-06-17,9
18129,21000,1967,chevrolet malibu,excellent,8,gas,94000.0,automatic,convertible,blue,False,2018-07-14,53
27143,1,1978,chevrolet suburban,good,8,gas,97500.0,automatic,SUV,blue,False,2019-01-27,13
18050,1,1978,chevrolet suburban,good,8,gas,97500.0,automatic,SUV,blue,False,2019-03-13,35
21210,14500,1980,chevrolet corvette,excellent,8,gas,68000.0,manual,convertible,custom,False,2018-06-15,35
24685,14500,1980,chevrolet corvette,excellent,8,gas,68000.0,manual,convertible,custom,False,2018-11-22,7
3973,11900,1981,chevrolet corvette,good,8,gas,96561.0,automatic,coupe,red,False,2018-12-23,53
28804,11900,1981,chevrolet corvette,good,8,gas,96561.0,automatic,coupe,red,False,2019-03-13,49


### Se realizó la limpieza y preparación de datos.
* Se cambiaron los datos nulos de las columnas 'model_year' y 'cylinders' por 2025 y 0 respectivamente, haciendo referencia a que desconocemos sus datos. Además, se cambió el tipo de variable de float a int.
* Se cambiaron los datos nulos de la columna 'odometer' por -1, haciendo referencia a que desconocemos el dato del kilometraje. No se colocó 0 como en las columnas anteriores, ya que pueden existir carros nuevos con 0 kilómetros recorridos.
* Se cambiaron los datos nulos de la columna 'paint_color' por 'unknown', ya que desconocemos el color del vehículo.
* Se cambiaron los datos nulos de la columna 'is_4wd' por 0, ya que solamente existían datos con el número 1 haciendo referencia a que sí son 4WD. Además, se cambiaron los datos que contenían 1 al valor booleano 'True' y los datos que contenían 0 por 'False' para tener un mejor entendimiento.
* ¿Por qué no se borraron los datos duplicados? Se evidenció que las filas duplicadas tenían una pequeña diferencia en las columnas de 'date_posted' y 'days_listed'. Se concluye que algunos usuarios publicaban nuevamente sus vehículos por diferentes razones, como error al momento de subir información o un posible intento de que el vehículo se venda más rápido.

## Análisis de datos

### Análisis por modelo de año

In [32]:
# Group the data by 'model_year' and sum the 'price'
price_per_model_year = vehicle_df.groupby('model_year')['price'].sum().reset_index()
price_per_model_year = price_per_model_year.sort_values(by='price', ascending=False)

fig = px.bar(price_per_model_year, x='model_year', y='price',
             title='Precio total por año de modelo del vehículo',
             labels={'model_year': 'Año de modelo', 'price': 'Precio total'},
             color='price',
             color_continuous_scale=px.colors.sequential.Plasma)
fig.show()
# Show the figure in Streamlit
# st.plotly_chart(fig, use_container_width=True) 

# Croup the data by 'model_year' and count the number of cars
cars_per_year = vehicle_df.groupby('model_year').size().reset_index(name='count')
cars_per_year = cars_per_year.sort_values(by='count', ascending=False)

fig = px.bar(
    cars_per_year,
    x='model_year',
    y='count',
    title='Cantidad de carros por año de modelo',
    labels={'model_year': 'Año de modelo', 'count': 'Cantidad de carros'},
    color='count',
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.show()


* Se observa una relación parcialmente directa entre el precio total de los vehículos por año y la cantidad de carros por año de modelo. La excepción más relevante son los datos desconocidos del año "2025", que corresponden a vehículos cuyo año de modelo no se conoce.

### Análisis por condición del vehículo

In [33]:
# Group the data by 'condition' and sum the 'price'
price_per_condition = vehicle_df.groupby('condition')['price'].sum().reset_index()
price_per_condition = price_per_condition.sort_values(by='price', ascending=False)

fig = px.bar(
    price_per_condition,
    x='condition',
    y='price',
    title='Cantidad de carros por condición',
    labels={'condition': 'Condición', 'price': 'Precio total'},
    color='price',
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.show()

* Los vehículos en mejor condición generan un mayor precio total en ventas. Esto indica que los autos en buen estado son más valorados y preferidos por los compradores, lo que se refleja en un mayor volumen de ventas para estas condiciones.

### Análisis de kilometraje ('odometer')

In [34]:
# Group the data by 'odometer' and mean of 'price'
price_per_odometer_mean = vehicle_df[vehicle_df['odometer'] >= 0].groupby('odometer')['price'].mean().reset_index()
price_per_odometer_mean = price_per_odometer_mean.sort_values(by='price', ascending=False)

fig = px.scatter(
    price_per_odometer_mean,
    x='odometer',
    y='price',
    title='Correlación entre kilometraje y precio medio',
    labels={'odometer': 'Kilometraje', 'price': 'Precio medio'},
    color_discrete_sequence=["#1F2DEA"]
)
fig.update_yaxes(title_text='Precio medio')
fig.show()
# Group the data by 'odometer' and sum of 'price'
price_per_odometer = vehicle_df[vehicle_df['odometer'] >= 0].groupby('odometer')['price'].sum().reset_index()
price_per_odometer = price_per_odometer.sort_values(by='price', ascending=False)

fig = px.scatter(
    price_per_odometer,
    x='odometer',
    y='price',
    title='Correlación entre kilometraje y precio total',
    labels={'odometer': 'Kilometraje', 'price': 'Precio total'},
    color_discrete_sequence=["#1F2DEA"]
)
fig.update_yaxes(title_text='Precio total')
fig.show()
# Correlation between 'odometer' and 'price'
correlation = vehicle_df[vehicle_df['odometer'] >= 0][['odometer', 'price']].corr().iloc[0, 1]

print(f"Correlación entre kilometraje y precio total': {correlation:.2f}")

Correlación entre kilometraje y precio total': -0.42


* Se observa que los vehículos con precios más elevados tienen, en promedio, un kilometraje entre 50,000 y 150,000 kilómetros. Sin embargo, existe una excepción importante representada por los datos desconocidos, identificados con el valor -1.

Además, al calcular la correlación entre el kilometraje y el precio, se obtuvo un valor de aproximadamente -0.42. Esto indica que existe una relación negativa débil, es decir, a mayor kilometraje, el precio tiende a ser menor, aunque la relación no es muy fuerte y existen otros factores que también influyen en el precio.

Además para este análisis **no se incluyeron los registros con valores desconocidos de kilometraje** (es decir, aquellos con odómetro menor a 0), con el fin de obtener resultados más precisos y representativos de la relación real entre estas variables.

### Análisis por tipo de vehículo

In [35]:
# Group the data by 'type' and sum the 'price'
price_per_type = vehicle_df.groupby('type')['price'].sum().reset_index()
price_per_type = price_per_type.sort_values(by='price', ascending=False)

fig = px.bar(
    price_per_type,
    x='type',
    y='price',
    title='Precio total por tipo de vehículo',
    labels={'type': 'Tipo', 'price': 'Precio total'},
    color='price',
    color_continuous_scale=px.colors.sequential.Plasma
)
fig.show()

# Group the data by 'type' and calculate the mean 'price'
price_per_type_mean = vehicle_df.groupby('type')['price'].mean().reset_index()
price_per_type_mean = price_per_type_mean.sort_values(by='price', ascending=False)

fig = px.bar(
    price_per_type_mean,
    x='type',
    y='price',
    title='Precio medio por tipo de vehículo',
    labels={'type': 'Tipo', 'price': 'Precio medio'},
    color='price',
    color_continuous_scale=px.colors.sequential.Viridis
)
fig.show()


* El tipo de vehículo 'truck' es el que genera el mayor precio total en ventas, mientras que el tipo 'bus' tiene el precio medio más alto. Esto indica que los 'truck' se venden en mayor cantidad, pero los 'bus', aunque menos frecuentes, se venden a precios promedio más elevados.

In [36]:
# Create a graph showing the distribution of 'type' using a bar chart 

type_counts = vehicle_df['type'].value_counts()
fig = px.bar(
    vehicle_df,
    x=type_counts.index,
    y=type_counts.values,
    title='Cantidad de vehículos por tipo',
    labels={'y': 'Cantidad', 'x': 'Tipo'},
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig.show()
# Create a graph showing the distribution of 'type'  with 'days_listed' using a line chart
type_per_days_listed = df_drop_dupli.groupby('days_listed')['type'].value_counts().unstack()
fig = px.line(
    type_per_days_listed,
    x=type_per_days_listed.index,
    y=type_per_days_listed.columns,
    title='Tipo de vehículo por días listados',
    labels={'days_listed': 'Días listados', 'value': 'Cantidad'},
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig.update_yaxes(title_text='Cantidad de vehículos')
fig.show()

* Se evidencia que los vehículos tipo 'sedan' y 'SUV' son los más ofertados y también los que se venden más rápido, ya que su cantidad disminuye rápidamente con los días listados. Esto indica alta demanda y rotación para estos tipos en la plataforma.

### Análisis de fecha de publicación y días listados

In [37]:
# Create a new column 'date_posted' with the current date and 'duplicados' DataFrame
cols = ['price', 'model_year', 'model', 'condition', 'odometer', 'fuel', 'transmission', 'paint_color']
duplicados['date_posted'] = pd.to_datetime(duplicados['date_posted'])
duplicados['min_date_posted'] = duplicados.groupby(cols)['date_posted'].transform('min')
duplicados['days_from_original'] = (duplicados['date_posted'] - duplicados['min_date_posted']).dt.days

duplicados.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,min_date_posted,days_from_original
10472,15900,1965,ford mustang,good,0,gas,60000.0,manual,convertible,black,False,2018-07-25,33,2018-07-25,0
34997,15900,1965,ford mustang,good,8,gas,60000.0,manual,convertible,black,False,2018-10-10,17,2018-07-25,77
42834,21000,1967,chevrolet malibu,excellent,8,gas,94000.0,automatic,convertible,blue,False,2018-06-17,9,2018-06-17,0
18129,21000,1967,chevrolet malibu,excellent,8,gas,94000.0,automatic,convertible,blue,False,2018-07-14,53,2018-06-17,27
27143,1,1978,chevrolet suburban,good,8,gas,97500.0,automatic,SUV,blue,False,2019-01-27,13,2019-01-27,0
18050,1,1978,chevrolet suburban,good,8,gas,97500.0,automatic,SUV,blue,False,2019-03-13,35,2019-01-27,45
21210,14500,1980,chevrolet corvette,excellent,8,gas,68000.0,manual,convertible,custom,False,2018-06-15,35,2018-06-15,0
24685,14500,1980,chevrolet corvette,excellent,8,gas,68000.0,manual,convertible,custom,False,2018-11-22,7,2018-06-15,160
3973,11900,1981,chevrolet corvette,good,8,gas,96561.0,automatic,coupe,red,False,2018-12-23,53,2018-12-23,0
28804,11900,1981,chevrolet corvette,good,8,gas,96561.0,automatic,coupe,red,False,2019-03-13,49,2018-12-23,80


In [38]:
# Filter the 'duplicados' DataFrame to only include rows where 'days_from_original' is greater than 0
duplicados_filtrados = duplicados[duplicados['days_from_original'] > 0]

fig = px.histogram(
    duplicados_filtrados,
    x='days_from_original',
    title='Días desde la fecha original de publicación para duplicados',
    labels={'days_from_original': 'Días desde la fecha original'},
    color_discrete_sequence=["#1F2DEA"]
)
fig.update_yaxes(title_text='Número de duplicados')
fig.show()


* La mayoría de los duplicados se publican nuevamente pocos días después de la publicación original, ya que la mayor parte de los registros tiene una baja cantidad de días desde la fecha original. Esto indica que los usuarios tienden a republicar sus vehículos rápidamente para aumentar sus posibilidades de venta o por algún error de información al momento de publicar el vehículo.

In [39]:
# Check the distribution of 'price' using a histogram
fig = px.histogram(
    vehicle_df,
    x='days_listed',
    title='Días listados',
    labels={'days_listed': 'Días listados del vehículo'},
    color_discrete_sequence=["#1F2DEA"]
)
fig.update_yaxes(title_text='Número de vehiculos')
fig.show()

* Se evidencia que la mayoría de los vehículos se venden en pocos días, ya que la mayor concentración de publicaciones tiene un bajo número de días listados. Esto indica que los autos suelen venderse rápidamente en la plataforma, mientras que solo un pequeño porcentaje permanece mucho tiempo publicada.

## Coprobación de hipótesis

1. Hipótesis central: Los vehículos en mejor condición se venden a precios más altos.
    * Hipótesis nula (H₀): Los vehículos se venden al misma precio sin importar la condición.
    * Hipótesis alternativa (H₁): Los vehículos en mejor condición se venden a precios más bajos.
2. Hipótesis central: Los vehículos tipo SUV y sedan se venden más rápido que otros tipos.
    * Hipótesis nula (H₀):  Los vehículos tipo SUV y sedan se venden igual de rápido que los otros.
    * Hipótesis alternativa (H₁): Los vehículos tipo SUV y sedan se venden menos rápido que otros tipos.
3. Hipótesis central: El kilometraje bajo está asociado a precios más elevados.
    * Hipótesis nula (H₀): El kilometraje bajo no está asociado a precios más elevados.
    * Hipótesis alternativa (H₁): El kilometraje bajo está asociado a precios más bajos.

In [40]:
# Hypothesis testing for first hypothesis
excellent_prices = vehicle_df[vehicle_df['condition'] == 'excellent']['price']
fair_prices = vehicle_df[vehicle_df['condition'] == 'fair']['price']


levene_statistic, levene_p_value = levene(excellent_prices, fair_prices)
print(f"Estadístico de Levene (W): {levene_statistic:.2f}")
print(f"Valor p: {levene_p_value:.2f}")

alpha = 0.05

results = st.ttest_ind(excellent_prices, fair_prices, equal_var=False)

print('Hipótesis nula (H₀): Los vehículos se venden al mismo precio sin importar la condición')
print('Valor p:', results.pvalue)

if results.pvalue < alpha:
    print("Rechazamos la hipótesis nula (los precios son diferentes según la condición)")
else:
    print("No podemos rechazar la hipótesis nula (los precios no son significativamente diferentes)")

Estadístico de Levene (W): 774.86
Valor p: 0.00
Hipótesis nula (H₀): Los vehículos se venden al mismo precio sin importar la condición
Valor p: 0.0
Rechazamos la hipótesis nula (los precios son diferentes según la condición)


* Se concluye que los vehículos con mejor condición se venden a precios mas altos.

In [41]:
# Hypothesis testing for second hypothesis
suv_sedan_days = vehicle_df[vehicle_df['type'].isin(['SUV', 'sedan'])]['days_listed']
others_days = vehicle_df[~vehicle_df['type'].isin(['SUV', 'sedan'])]['days_listed']

levene_statistic, levene_p_value = levene(suv_sedan_days, others_days)
print(f"Estadístico de Levene (W): {levene_statistic:.2f}")
print(f"Valor p: {levene_p_value:.2f}")

alpha = 0.05

results2 = st.ttest_ind(suv_sedan_days, others_days, equal_var=False)

print('Hipótesis nula (H₀): Los vehículos tipo SUV y sedan se venden igual de rápido que los otros')
print('Valor p:', results2.pvalue)

if results2.pvalue < alpha:
    print("Rechazamos la hipótesis nula (los vehículos SUV y sedan se venden más rápido que los otros)")
else:
    print("No podemos rechazar la hipótesis nula (los vehículos SUV y sedan no se venden más rápido que los otros)")

Estadístico de Levene (W): 0.02
Valor p: 0.88
Hipótesis nula (H₀): Los vehículos tipo SUV y sedan se venden igual de rápido que los otros
Valor p: 0.7008287299909921
No podemos rechazar la hipótesis nula (los vehículos SUV y sedan no se venden más rápido que los otros)


* Se concluye que los vehículos tipo 'SUV' y 'sedan' se venden parcialmente igual de rapidos que los otro tipos de vehículos.

In [42]:
# Hypothesis testing for the third hypothesis
low_mileage = vehicle_df[vehicle_df['odometer'] < 100000]['price']
high_mileage = vehicle_df[vehicle_df['odometer'] >= 100000]['price']

levene_statistic, levene_p_value = levene(low_mileage, high_mileage)
print(f"Estadístico de Levene (W): {levene_statistic:.2f}")
print(f"Valor p: {levene_p_value:.2f}")

alpha = 0.05

results2 = st.ttest_ind(low_mileage, high_mileage, equal_var=False)

print('Hipótesis nula (H₀): El kilometraje bajo no está asociado a precios más elevados.')
print('Valor p:', results2.pvalue)

if results2.pvalue < alpha:
    print("Rechazamos la hipótesis nula (el kilometraje bajo está asociado a precios más elevados)")
else:
    print("No podemos rechazar la hipótesis nula (el kilometraje bajo no está asociado a precios más elevados)")

Estadístico de Levene (W): 3785.24
Valor p: 0.00
Hipótesis nula (H₀): El kilometraje bajo no está asociado a precios más elevados.
Valor p: 0.0
Rechazamos la hipótesis nula (el kilometraje bajo está asociado a precios más elevados)


* Se concluye los vehículos con kilometraje bajo está asociado a precios más elevados.

# Conclusión

* Los vehículos en mejor condición se venden a precios significativamente más altos, lo que confirma la importancia del estado del auto en su valor de mercado.
* Los tipos de vehículos más ofertados y con mayor rotación son los 'SUV' y 'sedan', aunque estadísticamente se venden a una velocidad similar a otros tipos.
* Existe una relación negativa entre el kilometraje y el precio: a mayor kilometraje, menor es el precio promedio, aunque esta relación no es muy fuerte.
* La mayoría de los autos se venden en pocos días, lo que indica una alta demanda en la plataforma.
* Para todos los análisis estadísticos, se consideró la diferencia de varianzas entre grupos y se aplicaron las pruebas adecuadas según los resultados de Levene.
* Los registros con datos desconocidos (como kilometraje -1) no se incluyeron en los análisis de correlación para asegurar resultados más precisos.

## Recomendaciones

*  Mantener el vehículo en la mejor condición posible y documentar claramente su estado puede ayudar a obtener un mejor precio de venta.
* Priorizar vehículos con menor kilometraje y en mejor condición, ya que estos tienden a mantener mejor su valor.
* Mejorar la recolección de datos, incentivando a los usuarios a completar toda la información relevante del vehículo, especialmente el kilometraje y el año de modelo.
* Aprovechar la alta rotación de vehículos tipo 'SUV' y 'sedan', ya que son los más demandados y suelen venderse rápidamente.
* Considerar el impacto de otros factores como la ubicación, el historial de mantenimiento y el precio de mercado actual para obtener una visión más completa del comportamiento de ventas.

En resumen, el análisis permite identificar los factores clave que influyen en el precio y la velocidad de venta de los vehículos, proporcionando información útil tanto para vendedores como para compradores en la toma de decisiones dentro de la plataforma.